In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf

%matplotlib inline
plt.style.use('ggplot')

In [27]:
dataset_path = '../datasets/synergy-final-iter1-features/'

device = 'xdk_1'

df = pd.read_pickle(dataset_path + device + '.p')
df_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# filter activities
use_activities = [0,1,2,3,4,5,6,7]
df_labels = df_labels.loc[df_labels.label.isin(use_activities)]
df = df.loc[df.index.isin(df_labels.index)]

df = df.filter(regex='accel_.*index_mass_quantile')

# X = df.values
# y = df_labels['label'].values

In [28]:
labels = df_labels['label']

train_test_split = np.random.rand(len(df)) < 0.70
train_x = df[train_test_split]
train_y = labels[train_test_split]
test_x = df[~train_test_split]
test_y = labels[~train_test_split]

In [35]:
input_height = 1
input_width = 8
num_labels = len(use_activities)
num_channels = 3

batch_size = 10
kernel_size = 6
depth = 6
num_hidden = 1000

learning_rate = 0.0001
training_epochs = 8

total_batches = train_x.shape[0] // batch_size

In [36]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

In [38]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

c = apply_depthwise_conv(X,kernel_size,num_channels,depth)
p = apply_max_pool(c,3,2)
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10)

shape = c.get_shape().as_list()
c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])
f = tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))

out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])
y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)

ValueError: Negative dimension size caused by subtracting 6 from 1 for 'depthwise_8' (op: 'DepthwiseConv2dNative') with input shapes: [?,1,1,18], [1,6,18,0].

In [17]:
loss = -tf.reduce_sum(Y * tf.log(y_))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
cost_history = np.empty(shape=[1],dtype=float)

with tf.Session() as session:
    tf.initialize_all_variables().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            filtered_index = train_x.index[offset:(offset + batch_size)]
            batch_x = train_x.loc[train_x.index.isin(filtered_index)]
            batch_y = train_y.loc[train_y.index.isin(filtered_index)]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
        print("Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ",
              session.run(accuracy, feed_dict={X: train_x, Y: train_y}))

    print("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.


ValueError: Cannot feed value of shape (10, 24) for Tensor 'Placeholder_4:0', which has shape '(?, 1, 90, 3)'